<a href="https://colab.research.google.com/github/hikaru122700/kaggle-private/blob/%E5%9B%BD%E5%9C%9F%E4%BA%A4%E9%80%9A%E7%9C%81/002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!pip install optuna
!pip install catboost

In [30]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold
from itertools import combinations
import gc

from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import gc
gc.collect()
import optuna
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import log_loss
import time

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [31]:

# すべての列を表示するように設定
pd.set_option('display.max_columns', None)

In [32]:
# データの読み込み
# INPUT_DIRにtrain.csvなどのデータを置いているディレクトリを指定してください。

path  = "/content/drive/My Drive/signate/国土交通省/"

train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
sample_sub = pd.read_csv(path + "sample_submit.csv", header=None)

In [33]:
len(train), len(test)

(584507, 384540)

(584507, 384540)

In [34]:
test.head(3)

,index,target_ym,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,0,202207,a372446,1,2014-06-27 20:30:14,2024-02-09 00:10:05,3,シャーメゾンエランA,NaN,シャーメゾンELAN A棟,シャーメゾンエラン Aトウ,6.0,山口県山陽小野田市大字西高泊618-12,131.175432,34.007274,10.0,NaN,NaN,2.0,NaN,201101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210301/321001/210101/210401/320901,23270592,202,NaN,2.0,NaN,6.0,2.0,62.509998,250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340201/230103/310501/230201/230501/220501/3401...,37220460000756,2022-04-18 00:00:00,2022-04-18 00:00:00,2022-07-25 14:00:26,2022-08-01 00:00:00,1,1,3102,0.0,NaN,A0202,756.0,57.0,35,216,大字西高泊,618-12,NaN,122414490.0,472240000.0,JR山陽本線,小野田,高須,4.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,0,NaN,2.0,5.0,2,50,3000,3.0,NaN,NaN,NaN,鍵販売手数料,11000.0,安心サポート24プラス利用料月額,770.0,町内会費月額,500.0,NaN,2.0,1.0,NaN,NaN,1台無料 地上,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110102/110903/121002/210301/220101/220201/2203...,1.0,【個人契約】 初回契約事務手数料：22，000円（税込）、月額保証料：賃料等の1％,NaN,NaN
1,1,202301,a276097,1,2014-06-28 02:47:45,2024-08-15 02:24:34,1,マルコフォート,NaN,マルコフォート,マルコフオート,15.0,東京都調布市菊野台3丁目3-3,139.569165,35.653207,4.0,NaN,NaN,3.0,1.0,198703.0,NaN,NaN,18.0,20.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/321001/320901/210301/210201,853089,303,NaN,3.0,NaN,5.0,1.0,19.440001,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230801/240104/310501/263101/290101/230201/2605...,1038270025409,2023-01-30 00:00:00,2023-01-30 00:00:00,2023-01-30 16:44:45,2023-02-06 00:00:00,1,1,3101,0.0,2.0,303,182.0,7.0,13,208,菊野台３丁目,NaN,3-3,128339891.0,502460485.0,京王線,柴崎,NaN,NaN,240.0,京王線,つつじヶ丘,NaN,NaN,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,0,NaN,3.0,5.0,1,20,2000,3.0,NaN,NaN,NaN,消毒料,16500.0,入居安心サ

,index,target_ym,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,0,202207,a372446,1,2014-06-27 20:30:14,2024-02-09 00:10:05,3,シャーメゾンエランA,NaN,シャーメゾンELAN A棟,シャーメゾンエラン Aトウ,6.0,山口県山陽小野田市大字西高泊618-12,131.175432,34.007274,10.0,NaN,NaN,2.0,NaN,201101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210301/321001/210101/210401/320901,23270592,202,NaN,2.0,NaN,6.0,2.0,62.509998,250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340201/230103/310501/230201/230501/220501/3401...,37220460000756,2022-04-18 00:00:00,2022-04-18 00:00:00,2022-07-25 14:00:26,2022-08-01 00:00:00,1,1,3102,0.0,NaN,A0202,756.0,57.0,35,216,大字西高泊,618-12,NaN,122414490.0,472240000.0,JR山陽本線,小野田,高須,4.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,0,NaN,2.0,5.0,2,50,3000,3.0,NaN,NaN,NaN,鍵販売手数料,11000.0,安心サポート24プラス利用料月額,770.0,町内会費月額,500.0,NaN,2.0,1.0,NaN,NaN,1台無料 地上,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110102/110903/121002/210301/220101/220201/2203...,1.0,【個人契約】 初回契約事務手数料：22，000円（税込）、月額保証料：賃料等の1％,NaN,NaN
1,1,202301,a276097,1,2014-06-28 02:47:45,2024-08-15 02:24:34,1,マルコフォート,NaN,マルコフォート,マルコフオート,15.0,東京都調布市菊野台3丁目3-3,139.569165,35.653207,4.0,NaN,NaN,3.0,1.0,198703.0,NaN,NaN,18.0,20.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/321001/320901/210301/210201,853089,303,NaN,3.0,NaN,5.0,1.0,19.440001,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230801/240104/310501/263101/290101/230201/2605...,1038270025409,2023-01-30 00:00:00,2023-01-30 00:00:00,2023-01-30 16:44:45,2023-02-06 00:00:00,1,1,3101,0.0,2.0,303,182.0,7.0,13,208,菊野台３丁目,NaN,3-3,128339891.0,502460485.0,京王線,柴崎,NaN,NaN,240.0,京王線,つつじヶ丘,NaN,NaN,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,0,NaN,3.0,5.0,1,20,2000,3.0,NaN,NaN,NaN,消毒料,16500.0,入居安心サ

In [35]:
train.head(3)

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,202007,38000,a000002,1,2014-06-27 21:18:41,2024-05-25 00:45:07,3,Plaisirりんくう,NaN,プレジール りんくう,プレジールりんくう,22.0,大阪府泉佐野市笠松1丁目6-51,135.309302,34.411514,1.0,NaN,NaN,2.0,NaN,200210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321101/310201/320901/321001/210202/210301/210101,23810544,202,NaN,2.0,2.0,5.0,1.0,23.180000,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230801/310501/220301/290901/230101/240104/2205...,1344300002216,2020-07-30 00:00:00,2020-07-30 00:00:00,2020-07-30 16:21:04,2020-08-06 00:00:00,1,1,3102,0.0,2.0,202,598.0,44.0,27,213,笠松,NaN,1-6-51,123869683.0,487123547.0,南海線,泉佐野,NaN,0.0,1040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.18,0,NaN,2.0,5.0,1,20,3000.0,3.0,NaN,NaN,NaN,木下の賃貸 友の会費,1760.0,カギ交換代,16500.0,木下の消毒サービス,26400.0,8800.0,2.0,1.0,0.0,1.0,NaN,3.0,3,202009.0,NaN,NaN,NaN,NaN,NaN,NaN,239.0,1280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110702/210202/220101/220201/220301/230101/2308...,NaN,NaN,NaN,NaN
1,202107,103000,a000004,1,2017-02-05 20:43:26,2024-08-17 12:12:51,1,向原パークハイツ,NaN,向原パークハイツ,ムカイハラパークハイツ,NaN,東京都板橋区向原1丁目11-1,139.684556,35.741834,4.0,NaN,NaN,3.0,NaN,197612.0,NaN,NaN,44.099998,44.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/330501/320901/321001/210301/210201,33983775,110,NaN,1.0,4.0,5.0,2.0,47.700001,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340101/253401/220401/290901/220201/230202/2943...,1279070015508,2021-07-09 00:00:00,2021-07-09 00:00:00,2021-07-30 23:36:05,2021-08-06 00:00:00,1,0,3101,0.0,NaN,110,173.0,36.0,13,119,向原１丁目,11-1,NaN,128659541.0,502875929.0,東京メトロ有楽町線,小竹向原,NaN,NaN,320.0,東京メトロ有楽町線,千川,NaN,NaN,400.0,東京地下鉄副都心線 小竹向原 徒歩4分,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.70,0,NaN,1.0,5.0,2,30,6

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,202007,38000,a000002,1,2014-06-27 21:18:41,2024-05-25 00:45:07,3,Plaisirりんくう,NaN,プレジール りんくう,プレジールりんくう,22.0,大阪府泉佐野市笠松1丁目6-51,135.309302,34.411514,1.0,NaN,NaN,2.0,NaN,200210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321101/310201/320901/321001/210202/210301/210101,23810544,202,NaN,2.0,2.0,5.0,1.0,23.180000,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230801/310501/220301/290901/230101/240104/2205...,1344300002216,2020-07-30 00:00:00,2020-07-30 00:00:00,2020-07-30 16:21:04,2020-08-06 00:00:00,1,1,3102,0.0,2.0,202,598.0,44.0,27,213,笠松,NaN,1-6-51,123869683.0,487123547.0,南海線,泉佐野,NaN,0.0,1040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.18,0,NaN,2.0,5.0,1,20,3000.0,3.0,NaN,NaN,NaN,木下の賃貸 友の会費,1760.0,カギ交換代,16500.0,木下の消毒サービス,26400.0,8800.0,2.0,1.0,0.0,1.0,NaN,3.0,3,202009.0,NaN,NaN,NaN,NaN,NaN,NaN,239.0,1280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110702/210202/220101/220201/220301/230101/2308...,NaN,NaN,NaN,NaN
1,202107,103000,a000004,1,2017-02-05 20:43:26,2024-08-17 12:12:51,1,向原パークハイツ,NaN,向原パークハイツ,ムカイハラパークハイツ,NaN,東京都板橋区向原1丁目11-1,139.684556,35.741834,4.0,NaN,NaN,3.0,NaN,197612.0,NaN,NaN,44.099998,44.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/330501/320901/321001/210301/210201,33983775,110,NaN,1.0,4.0,5.0,2.0,47.700001,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340101/253401/220401/290901/220201/230202/2943...,1279070015508,2021-07-09 00:00:00,2021-07-09 00:00:00,2021-07-30 23:36:05,2021-08-06 00:00:00,1,0,3101,0.0,NaN,110,173.0,36.0,13,119,向原１丁目,11-1,NaN,128659541.0,502875929.0,東京メトロ有楽町線,小竹向原,NaN,NaN,320.0,東京メトロ有楽町線,千川,NaN,NaN,400.0,東京地下鉄副都心線 小竹向原 徒歩4分,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.70,0,NaN,1.0,5.0,2,30,6

In [9]:
all_df = pd.concat([train, test])
all_df.drop("index", axis=1, inplace=True)

In [10]:
all_df.head(3)

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,202007,38000.0,a000002,1,2014-06-27 21:18:41,2024-05-25 00:45:07,3,Plaisirりんくう,NaN,プレジール りんくう,プレジールりんくう,22.0,大阪府泉佐野市笠松1丁目6-51,135.309302,34.411514,1.0,NaN,NaN,2.0,NaN,200210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,321101/310201/320901/321001/210202/210301/210101,23810544,202,NaN,2.0,2.0,5.0,1.0,23.180000,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230801/310501/220301/290901/230101/240104/2205...,1344300002216,2020-07-30 00:00:00,2020-07-30 00:00:00,2020-07-30 16:21:04,2020-08-06 00:00:00,1,1,3102,0.0,2.0,202,598.0,44.0,27,213,笠松,NaN,1-6-51,123869683.0,487123547.0,南海線,泉佐野,NaN,0.0,1040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.18,0,NaN,2.0,5.0,1,20,3000.0,3.0,NaN,NaN,NaN,木下の賃貸 友の会費,1760.0,カギ交換代,16500.0,木下の消毒サービス,26400.0,8800.0,2.0,1.0,0.0,1.0,NaN,3.0,3,202009.0,NaN,NaN,NaN,NaN,NaN,NaN,239.0,1280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110702/210202/220101/220201/220301/230101/2308...,NaN,NaN,NaN,NaN
1,202107,103000.0,a000004,1,2017-02-05 20:43:26,2024-08-17 12:12:51,1,向原パークハイツ,NaN,向原パークハイツ,ムカイハラパークハイツ,NaN,東京都板橋区向原1丁目11-1,139.684556,35.741834,4.0,NaN,NaN,3.0,NaN,197612.0,NaN,NaN,44.099998,44.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/330501/320901/321001/210301/210201,33983775,110,NaN,1.0,4.0,5.0,2.0,47.700001,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340101/253401/220401/290901/220201/230202/2943...,1279070015508,2021-07-09 00:00:00,2021-07-09 00:00:00,2021-07-30 23:36:05,2021-08-06 00:00:00,1,0,3101,0.0,NaN,110,173.0,36.0,13,119,向原１丁目,11-1,NaN,128659541.0,502875929.0,東京メトロ有楽町線,小竹向原,NaN,NaN,320.0,東京メトロ有楽町線,千川,NaN,NaN,400.0,東京地下鉄副都心線 小竹向原 徒歩4分,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.70,0,NaN,1.0,5.0,2,

In [11]:
# ユニークな値の数を取得
unique_count = all_df["money_hoshou_company"].nunique(dropna=True)
print(f"ユニークな building_name の数: {unique_count}")


ユニークな building_name の数: 191552


In [12]:
type("money_hoshou_company")

str

In [13]:

# 1. 'full_address' 列の先頭3文字を抽出して 'prefecture' 列に追加
all_df["prefecture"] = all_df["full_address"].str[:3]

# 2. LabelEncoder のインスタンスを作成
le = LabelEncoder()

# 3. 'prefecture' 列にラベルエンコーディングを適用し、新しい列 'prefecture_encoded' に保存
all_df["prefecture_encoded"] = le.fit_transform(all_df["prefecture"])
all_df["prefecture_encoded"]

,prefecture_encoded
0,9
1,26
2,4
3,21
4,21
...,...
384535,25
384536,21
384537,26
384538,21


In [14]:
for x in all_df.columns:
    if all_df[x].dtype == 'object' or "ruby" in x or "id" in x:
        print(x)
        print(all_df[x].head(1))
        all_df.drop(x, axis=1, inplace=True)


building_id
0    a000002
Name: building_id, dtype: object
building_create_date
0    2014-06-27 21:18:41
Name: building_create_date, dtype: object
building_modify_date
0    2024-05-25 00:45:07
Name: building_modify_date, dtype: object
building_name
0    Plaisirりんくう
Name: building_name, dtype: object
building_name_ruby
0   NaN
Name: building_name_ruby, dtype: float64
homes_building_name
0    プレジール りんくう
Name: homes_building_name, dtype: object
homes_building_name_ruby
0    プレジールりんくう
Name: homes_building_name_ruby, dtype: object
full_address
0    大阪府泉佐野市笠松1丁目6-51
Name: full_address, dtype: object
land_seigen
0    NaN
Name: land_seigen, dtype: object
reform_exterior
0    NaN
Name: reform_exterior, dtype: object
reform_exterior_other
0    NaN
Name: reform_exterior_other, dtype: object
reform_common_area
0    NaN
Name: reform_common_area, dtype: object
building_tag_id
0    321101/310201/320901/321001/210202/210301/210101
Name: building_tag_id, dtype: object
unit_id
0    23810544
Name: unit_id

In [15]:
all_df

,target_ym,money_room,building_status,building_type,unit_count,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,building_area_kind,management_form,management_association_flg,reform_exterior_date,reform_common_area_date,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_wet_area_date,reform_interior_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,post1,post2,addr1_1,addr1_2,nl,el,bus_time1,walk_distance1,bus_time2,walk_distance2,traffic_car,snapshot_land_area,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota1,money_sonota2,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,genkyo_code,usable_status,usable_date,school_ele_distance,school_ele_code,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_distance,parking_keiyaku,free_rent_duration,free_rent_gen_timing,prefecture_encoded
0,202007,38000.0,1,3,22.0,135.309302,34.411514,1.0,NaN,NaN,2.0,NaN,200210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,5.0,1.0,23.180000,120.0,NaN,NaN,NaN,1,1,3102,0.0,2.0,598.0,44.0,27,213,123869683.0,487123547.0,0.0,1040.0,NaN,NaN,NaN,NaN,NaN,NaN,23.18,0,NaN,2.0,5.0,1,20,3000.0,3.0,NaN,NaN,NaN,1760.0,16500.0,26400.0,8800.0,2.0,1.0,0.0,1.0,3.0,3,202009.0,NaN,NaN,NaN,NaN,239.0,1280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,202107,103000.0,1,1,NaN,139.684556,35.741834,4.0,NaN,NaN,3.0,NaN,197612.0,NaN,NaN,44.099998,44.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,5.0,2.0,47.700001,230.0,NaN,NaN,NaN,1,0,3101,0.0,NaN,173.0,36.0,13,119,128659541.0,502875929.0,NaN,320.0,NaN,400.0,NaN,NaN,NaN,NaN,47.70,0,NaN,1.0,5.0,2,30,6000.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,22000.0,2.0,3.0,300.0,NaN,2.0,1,NaN,291.0,NaN,340.0,NaN,399.0,434.0,NaN,NaN,435.0,NaN,NaN,320.0,NaN,NaN,NaN,26
2,202101,50000.0,1,3,NaN,141.352789,43.087834,1.0,NaN,NaN,3.0,NaN,199607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,1.0,NaN,NaN,2.0,NaN,3.0,2.0,46.349998,250.0,NaN,NaN,NaN,1,0,3102,NaN,NaN,65.0,21.0,1,103,155106911.0,508882127.0,NaN,880.0,NaN,1120.0,NaN,NaN,NaN,NaN,46.35,0,NaN,2.0,3.0,2,50,1500.0,2.0,NaN,NaN,NaN,200.0,33000.0,NaN,8800.0,2.0,1.0,0.0,NaN,NaN,1,NaN,538.0,NaN,644.0,NaN,201.0,741.0,574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,202007,37000.0,1,3,NaN,133.335824,34.502380,3.0,NaN,NaN,2.0,NaN,199309.0,NaN,NaN,NaN,NaN,NaN,12.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,6.0,2.0,39.740002,230.0,NaN,NaN,NaN,1,1,3102,NaN,2.0,720.0,92.0,34,207,124196872.0,480018303.0,25.0,700.0,15.0,700.0,NaN,NaN,NaN,NaN,39.74,0,NaN,1.0,6.0,2,30,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3300.0,2.0,1.0,NaN,7.0,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,1114.0,NaN,NaN,NaN,NaN,NaN,1114.0,NaN,NaN,NaN,21
4,202101,33000.0,1,3,NaN,133.335824,34.502380,3.0,NaN,NaN,2.0,NaN,199309.0,NaN,NaN,NaN,NaN,NaN,12.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,6.0,2.0,39.740002,230.0,NaN,NaN,NaN,1,1,3102,NaN,NaN,720.0,92.0,34,207,124196970.0,480018256.0,NaN,2012.0,NaN,3532.0,NaN,NaN,NaN,NaN,39.74,0,NaN,2.0,7.0,2,30,2000.0,NaN,NaN,NaN,NaN,33000.0,NaN,NaN,3300.0,2.0,1.0,0.0,NaN,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,469.0,NaN,NaN,NaN,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [16]:
all_df.columns

Index(['target_ym', 'money_room', 'building_status', 'building_type',
       'unit_count', 'lon', 'lat', 'building_structure', 'total_floor_area',
       'building_area', 'floor_count', 'basement_floor_count', 'year_built',
       'building_land_area', 'land_area_all', 'unit_area_min', 'unit_area_max',
       'building_land_chimoku', 'land_youto', 'land_toshi', 'land_chisei',
       'land_area_kind', 'land_setback_flg', 'land_setback', 'land_kenpei',
       'land_youseki', 'land_road_cond', 'building_area_kind',
       'management_form', 'management_association_flg', 'reform_exterior_date',
       'reform_common_area_date', 'room_floor', 'balcony_area',
       'dwelling_unit_window_angle', 'room_count', 'unit_area',
       'floor_plan_code', 'reform_date', 'reform_wet_area_date',
       'reform_interior_date', 'flg_open', 'flg_own', 'bukken_type',
       'flg_investment', 'empty_number', 'post1', 'post2', 'addr1_1',
       'addr1_2', 'nl', 'el', 'bus_time1', 'walk_distance1', 'bus_time

In [17]:
train = all_df[:len(train)]
test = all_df[len(train):]


In [18]:
# 目的変数と説明変数に分割
train_x = train.drop("money_room", axis=1)
train_y = train["money_room"]
test_x = test.drop("money_room", axis=1)

In [19]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [20]:
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import time

def evaluate(features):
    dtrain = xgb.DMatrix(tr_x[features], label=tr_y)
    dvalid = xgb.DMatrix(va_x[features], label=va_y)
    params = {'verbosity': 0, 'random_state': 71}
    num_round = 10  # 実際にはもっと多いround数が適切
    early_stopping_rounds = 3
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round,
                      evals=watchlist, early_stopping_rounds=early_stopping_rounds,
                      verbose_eval=0)
    va_pred = model.predict(dvalid)

    # RMSEを計算
    rmse = mean_squared_error(va_y, va_pred, squared=False)
    return rmse

def objective(trial):
    selected_features = []
    for col in train_x.columns:
        use_feature = trial.suggest_categorical(col, [True, False])
        if use_feature:
            selected_features.append(col)

    if len(selected_features) == 0:
        return float('inf')  # RMSEで全く特徴量なしの場合は非常に大きな値を返す

    score = evaluate(selected_features)
    return score

# Optunaによる最適化（RMSEは小さいほど良いので direction='minimize'）
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_trial = study.best_trial

print("Best Score (RMSE):", best_trial.value)
print("Best Feature Set:")
best_features = [k for k, v in best_trial.params.items() if v == True]
print(best_features)


[I 2024-12-10 04:32:53,060] A new study created in memory with name: no-name-938cf16e-f16a-4e15-97ad-9cfceb5cc222
[I 2024-12-10 04:32:56,572] Trial 0 finished with value: 20635.768667762317 and parameters: {'target_ym': True, 'building_status': False, 'building_type': False, 'unit_count': False, 'lon': False, 'lat': True, 'building_structure': True, 'total_floor_area': True, 'building_area': True, 'floor_count': True, 'basement_floor_count': True, 'year_built': True, 'building_land_area': False, 'land_area_all': True, 'unit_area_min': True, 'unit_area_max': True, 'building_land_chimoku': True, 'land_youto': False, 'land_toshi': False, 'land_chisei': False, 'land_area_kind': True, 'land_setback_flg': True, 'land_setback': True, 'land_kenpei': True, 'land_youseki': True, 'land_road_cond': True, 'building_area_kind': True, 'management_form': True, 'management_association_flg': False, 'reform_exterior_date': False, 'reform_common_area_date': True, 'room_floor': True, 'balcony_area': False,

Best Score (RMSE): 14985.424603936828
Best Feature Set:
['target_ym', 'building_status', 'building_type', 'unit_count', 'lat', 'building_area', 'floor_count', 'year_built', 'land_area_all', 'unit_area_min', 'building_land_chimoku', 'land_chisei', 'land_area_kind', 'land_setback', 'management_form', 'reform_exterior_date', 'dwelling_unit_window_angle', 'room_count', 'unit_area', 'floor_plan_code', 'reform_interior_date', 'flg_open', 'bukken_type', 'post1', 'post2', 'addr1_2', 'el', 'bus_time2', 'walk_distance2', 'traffic_car', 'snapshot_land_area', 'land_mochibun_a', 'land_mochibun_b', 'house_area', 'flg_new', 'room_kaisuu', 'madori_kind_all', 'money_kyoueki_tax', 'money_rimawari_now', 'money_shuuzen', 'money_sonota1', 'money_sonota3', 'parking_money_tax', 'parking_kubun', 'parking_distance', 'usable_status', 'usable_date', 'school_ele_distance', 'school_jun_distance', 'convenience_distance', 'shopping_street_distance', 'est_other_distance', 'free_rent_duration', 'free_rent_gen_timing',

In [21]:
# @title デフォルトのタイトル テキスト
from sklearn.linear_model import LogisticRegression

import numpy as np
import gc
from itertools import combinations
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from itertools import combinations
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error  # r2_score から mean_squared_error に変更
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

def get_models_trained(train, test, target, num_folds, train_eval):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)

    oof_predictions = np.zeros(len(train))
    test_predictions = np.zeros(len(test))
    train_eval_predictions = np.zeros(len(train_eval))

    # 評価指標の累計を保持する辞書
    sum_metric = {}
    # 各組み合わせの選択回数を保持する辞書
    combination_count = {}

    # 個別モデルの名前リスト
    model_names = ["model1", "model2", "model3", "model4"]

    # 全ての組み合わせを生成（1, 2, 3, 4モデル）
    all_combinations = []
    for r in range(1, 5):
        all_combinations.extend(combinations(model_names, r))

    # 初期化
    for comb in all_combinations:
        sum_metric[comb] = 0
        combination_count[comb] = 0

    for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
        print(f"Starting Fold {fold + 1}")
        X_train, X_valid = train[train_index], train[valid_index]
        y_train, y_valid = target[train_index], target[valid_index]

        # モデルと評価指標の初期化
        metric_dict = {}
        valid_pred_dict = {}

        # モデル1: XGBRegressor
        params1 = {
            "n_estimators": 626,
            "max_depth": 3,
            "random_state": 13,
            "min_child_weight": 0.001190123543553736,
            "learning_rate": 0.010519736270936835,
            "subsample": 0.7304788478701394,
            "colsample_bylevel": 0.604447278915981,
            "colsample_bytree": 0.7616852136157319,
            "reg_alpha": 0.115175569924065,
            "reg_lambda": 0.07155347824929895
        }
        model1 = XGBRegressor(**params1)

        # モデル2: CatBoostRegressor
        params2 = {
            'iterations': 254,
            'depth': 5,
            'learning_rate': 0.08377009991199288,
            'l2_leaf_reg': 1,
            'bagging_temperature': 0.7204457890870082,
            'min_data_in_leaf': 4,
            "random_state": 13,
            "verbose": 0
        }
        model2 = CatBoostRegressor(**params2)

        # モデル3: XGBRegressor
        params3 = {
            "n_estimators": 897,
            "max_depth": 4,
            "min_child_weight": 1.9636282677053687,
            "learning_rate": 0.006151391207761763,
            "subsample": 0.8251910979922186,
            "colsample_bylevel": 0.6454805596196158,
            "colsample_bytree": 0.598042694363472,
            "reg_alpha": 2.1719824223479005,
            "reg_lambda": 0.009192236594241635,
        }
        model3 = XGBRegressor(**params3)

        # モデル4: CatBoostRegressor
        params4 = {
            "iterations": 940,
            "depth": 3,
            "learning_rate": 0.019908189422344794,
            "l2_leaf_reg": 1,
            "bagging_temperature": 0.5063893392618839,
            "min_data_in_leaf": 1,
            'random_state': 42
        }
        model4 = CatBoostRegressor(**params4, verbose=0)

        # 各モデルの訓練と予測
        models = {
            "model1": model1,
            "model2": model2,
            "model3": model3,
            "model4": model4
        }

        for name, model in models.items():
            model.fit(X_train, y_train)
            preds = model.predict(X_valid)
            rmse = mean_squared_error(y_valid, preds, squared=False)  # RMSEを計算
            print(f"Fold {fold + 1} RMSE for {name} = {rmse}")
            metric_dict[(name,)] = rmse  # キーをタプルに変更
            valid_pred_dict[(name,)] = preds
            sum_metric[comb] += rmse  # 各組み合わせの累積RMSEを更新

        # すべての2モデルおよび3モデルの組み合わせを評価
        for r in [2, 3]:
            for comb in combinations(model_names, r):
                # 平均予測
                preds_comb = np.mean([valid_pred_dict[(comb_part,)] for comb_part in comb], axis=0)
                rmse_comb = mean_squared_error(y_valid, preds_comb, squared=False)  # RMSEを計算
                print(f"Fold {fold + 1} RMSE for {comb} = {rmse_comb}")
                metric_dict[comb] = rmse_comb
                valid_pred_dict[comb] = preds_comb
                sum_metric[comb] += rmse_comb

        # 全モデルのアンサンブル
        preds_all = np.mean([valid_pred_dict[(model,)] for model in model_names], axis=0)
        rmse_all = mean_squared_error(y_valid, preds_all, squared=False)  # RMSEを計算
        print(f"Fold {fold + 1} RMSE for all models ensemble = {rmse_all}")
        metric_dict[tuple(model_names)] = rmse_all
        valid_pred_dict[tuple(model_names)] = preds_all
        sum_metric[tuple(model_names)] += rmse_all

        # 最適な組み合わせを選択（RMSEが低い組み合わせを選択）
        best_combination = min(metric_dict, key=metric_dict.get)
        best_rmse = metric_dict[best_combination]
        print(f"Fold {fold + 1} best combination: {best_combination} with RMSE = {best_rmse}")

        # OOF予測とテスト予測の更新
        if isinstance(best_combination, tuple):
            # 複数モデルのアンサンブル
            oof_predictions[valid_index] = valid_pred_dict[best_combination]
            test_preds = np.mean([models[model].predict(test) for model in best_combination], axis=0)
            train_eval_preds = np.mean([models[model].predict(train_eval) for model in best_combination], axis=0)
        else:
            # 単一モデル（この場合は常にタプルになるため不要）
            oof_predictions[valid_index] = valid_pred_dict[(best_combination,)]
            test_preds = models[best_combination].predict(test)
            train_eval_preds = models[best_combination].predict(train_eval)

        # 各組み合わせの選択回数をインクリメント
        combination_count[best_combination] += 1

        test_predictions += test_preds / kf.n_splits
        train_eval_predictions += train_eval_preds / kf.n_splits

        # メモリの解放
        del X_train, X_valid, y_train, y_valid, model1, model2, model3, model4
        gc.collect()

        print('---------------\n')

    # 全体のRMSEを計算
    final_rmse = mean_squared_error(target, oof_predictions, squared=False)
    print(f"OOF RMSE = {final_rmse}")

    # 各組み合わせの累積評価指標と選択回数を表示
    print("Sum of RMSE for each combination:")
    for comb, metric in sum_metric.items():
        print(f"{comb}: {metric}")

    print("\nNumber of times each combination was selected:")
    for comb, count in combination_count.items():
        print(f"{comb}: {count}")

    return oof_predictions, test_predictions, train_eval_predictions

# 使用例（回帰データを用いる場合）
# oof_predictions, test_preds, train_preds = get_models_trained(X, X_test, y, 5, X)

In [22]:
train[list(best_features)]

,target_ym,building_status,building_type,unit_count,lat,building_area,floor_count,year_built,land_area_all,unit_area_min,building_land_chimoku,land_chisei,land_area_kind,land_setback,management_form,reform_exterior_date,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_interior_date,flg_open,bukken_type,post1,post2,addr1_2,el,bus_time2,walk_distance2,traffic_car,snapshot_land_area,land_mochibun_a,land_mochibun_b,house_area,flg_new,room_kaisuu,madori_kind_all,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_sonota1,money_sonota3,parking_money_tax,parking_kubun,parking_distance,usable_status,usable_date,school_ele_distance,school_jun_distance,convenience_distance,shopping_street_distance,est_other_distance,free_rent_duration,free_rent_gen_timing,prefecture_encoded
0,202007,1,3,22.0,34.411514,NaN,2.0,200210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,5.0,1.0,23.180000,120.0,NaN,1,3102,598.0,44.0,213,487123547.0,NaN,NaN,NaN,NaN,NaN,NaN,23.18,0,2.0,20,3.0,NaN,NaN,1760.0,26400.0,2.0,1.0,0.0,3,202009.0,NaN,NaN,239.0,NaN,NaN,NaN,NaN,9
1,202107,1,1,NaN,35.741834,NaN,3.0,197612.0,NaN,44.099998,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,47.700001,230.0,NaN,1,3101,173.0,36.0,119,502875929.0,NaN,400.0,NaN,NaN,NaN,NaN,47.70,0,1.0,30,3.0,NaN,NaN,NaN,NaN,2.0,3.0,300.0,1,NaN,291.0,340.0,399.0,NaN,320.0,NaN,NaN,26
2,202101,1,3,NaN,43.087834,NaN,3.0,199607.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,2.0,46.349998,250.0,NaN,1,3102,65.0,21.0,103,508882127.0,NaN,1120.0,NaN,NaN,NaN,NaN,46.35,0,2.0,50,2.0,NaN,NaN,200.0,NaN,2.0,1.0,0.0,1,NaN,538.0,644.0,201.0,NaN,NaN,NaN,NaN,4
3,202007,1,3,NaN,34.502380,NaN,2.0,199309.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,39.740002,230.0,NaN,1,3102,720.0,92.0,207,480018303.0,15.0,700.0,NaN,NaN,NaN,NaN,39.74,0,1.0,30,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,1,NaN,NaN,NaN,NaN,NaN,1114.0,NaN,NaN,21
4,202101,1,3,NaN,34.502380,NaN,2.0,199309.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,39.740002,230.0,NaN,1,3102,720.0,92.0,207,480018256.0,NaN,3532.0,NaN,NaN,NaN,NaN,39.74,0,2.0,30,NaN,NaN,NaN,33000.0,NaN,2.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,469.0,NaN,NaN,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584502,202001,1,3,NaN,34.713620,NaN,2.0,201202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,57.470001,250.0,NaN,1,3102,431.0,431.0,221,495175024.0,NaN,NaN,NaN,NaN,NaN,NaN,57.47,0,2.0,50,2.0,NaN,NaN,1150.0,NaN,2.0,1.0,0.0,3,202002.0,463.0,365.0,283.0,NaN,NaN,NaN,NaN,42
584503,202107,1,3,NaN,43.813140,NaN,2.0,201501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,69.139999,350.0,NaN,1,3102,79.0,8413.0,204,512804890.0,NaN,9920.0,NaN,NaN,NaN,NaN,69.14,0,2.0,50,3.0,NaN,NaN,500.0,NaN,2.0,1.0,NaN,1,NaN,NaN,NaN,1032.0,NaN,NaN,NaN,NaN,4
584504,202001,1,1,NaN,35.737079,NaN,4.0,199403.0,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,41.049999,150.0,NaN,1,3101,116.0,13.0,118,503183080.0,NaN,400.0,NaN,NaN,NaN,NaN,41.05,0,4.0,50,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1,NaN,NaN,NaN,180.0,NaN,NaN,NaN,NaN,26
584505,201907,1,3,4.0,43.786349,NaN,2.0,199610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,51.840000,250.0,NaN,1,3102,79.0,8415.0,204,512660281.0,NaN,NaN,NaN,NaN,NaN,NaN,51.84,0,2.0,50,2.0,NaN,NaN,NaN,1000.0,NaN,1.0,0.0,1,NaN,1092.0,1843.0,NaN,NaN,NaN,NaN,NaN,4


In [23]:
# 目的変数と説明変数に分割
X = train[list(best_features)].values
y = train["money_room"].values
X_test = test[list(best_features)].values

In [24]:
oof_predictions, test_preds, train_preds = get_models_trained(X, X_test, y, 5, X)

Starting Fold 1
Fold 1 RMSE for model1 = 14605.15157588933
Fold 1 RMSE for model2 = 15322.759558618534
Fold 1 RMSE for model3 = 13685.901447820273
Fold 1 RMSE for model4 = 13624.053216240669
Fold 1 RMSE for ('model1', 'model2') = 13479.12137287937
Fold 1 RMSE for ('model1', 'model3') = 14070.110659238679
Fold 1 RMSE for ('model1', 'model4') = 13933.057752570247
Fold 1 RMSE for ('model2', 'model3') = 13209.326767012752
Fold 1 RMSE for ('model2', 'model4') = 13301.025249077424
Fold 1 RMSE for ('model3', 'model4') = 13514.768604711207
Fold 1 RMSE for ('model1', 'model2', 'model3') = 13254.612793459786
Fold 1 RMSE for ('model1', 'model2', 'model4') = 13238.093589278926
Fold 1 RMSE for ('model1', 'model3', 'model4') = 13795.16479626289
Fold 1 RMSE for ('model2', 'model3', 'model4') = 13032.031230963052
Fold 1 RMSE for all models ensemble = 13203.496802422716
Fold 1 best combination: ('model2', 'model3', 'model4') with RMSE = 13032.031230963052
---------------

Starting Fold 2
Fold 2 RMSE fo

In [25]:
sample_sub

,0,1
0,0,25310
1,1,70766
2,2,97844
3,3,84854
4,4,18753
...,...,...
384535,384535,72137
384536,384536,59275
384537,384537,81590
384538,384538,53852


In [26]:
# 予測結果を提出用のフォーマットに格納
sample_sub.iloc[:, 1] = test_preds
sample_sub

,0,1
0,0,61638.241143
1,1,53243.461157
2,2,60433.200774
3,3,49593.164736
4,4,80231.207338
...,...,...
384535,384535,63879.271778
384536,384536,79414.152745
384537,384537,87898.932493
384538,384538,57756.953763


In [27]:
sample_sub.to_csv("国土交通省_002.csv", index=False, header=False)